In [4]:
from ogb.linkproppred import LinkPropPredDataset
import pandas as pd

In [5]:
dataset = LinkPropPredDataset(name = 'ogbl-ddi')


  0%|          | 0/46 [00:00<?, ?it/s]

Downloaded 0.04 GB: 100%|██████████| 46/46 [00:16<00:00,  2.83it/s]


Extracting dataset/ddi.zip
Loading necessary files...
This might take a while.


100%|██████████| 1/1 [00:00<00:00, 24.86it/s]

Processing graphs...
Saving...


In [7]:
split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0] # graph: library-agnostic graph object

In [13]:
df = pd.DataFrame(graph['edge_index']).T
df.columns = ['start', 'end']
df.head()

,start,end
0,4039,2424
1,2424,4039
2,4039,225
3,225,4039
4,4039,3901


In [21]:
len(set(df.start)) == len(set(df.end))

True